In [1]:
import torch
import theseus as th

torch.set_default_dtype(torch.float32)

In [2]:
def pose_error(optim_vars, aux_vars):
    pass

In [3]:
gt_poses = [
    th.SE2(torch.tensor([[0.,0,0]]),name='gt_start'),
    th.SE2(torch.tensor([[1.,0,0]]),name='gt_finish')
]
gt_poses

[SE2(tensor=tensor([[0., 0., 1., 0.]]), name=gt_start),
 SE2(tensor=tensor([[1., 0., 1., 0.]]), name=gt_finish)]

In [4]:
for g in gt_poses:
    print(g)

SE2(xytheta=tensor([[0., 0., 0.]]), name=gt_start)
SE2(xytheta=tensor([[1., 0., 0.]]), name=gt_finish)


In [5]:
pred_poses =[th.rand_se2(1), th.rand_se2(1)]
# pred_poses = gt_poses.copy()
pred_poses[0].name = "SE2_1"
pred_poses[1].name = "SE2_2"
pred_poses 

[SE2(tensor=tensor([[ 0.5052,  0.0529,  0.2942, -0.9557]]), name=SE2_1),
 SE2(tensor=tensor([[ 0.1407,  0.2569,  0.6128, -0.7902]]), name=SE2_2)]

In [28]:
measurement = gt_poses[0].between(gt_poses[1])
measurement

SE2(tensor=tensor([[1., 0., 1., 0.]]), name=SE2__2724)

In [7]:
measurement.name = "Odo_1"
measurement

SE2(tensor=tensor([[1., 0., 1., 0.]]), name=Odo_1)

In [8]:
odo_w_1 = th.ScaleCostWeight(1.0, name="odo_weight_1")
odo_w_1.name

'odo_weight_1'

In [9]:
odo_cost_f = th.Between(pred_poses[0], pred_poses[1], measurement, cost_weight=odo_w_1, name='odo_cf')

In [10]:
odo_cost_f.error()

tensor([[-1.3401, -0.0502,  0.3610]])

In [11]:
objective = th.Objective()

In [12]:
# def quad_error_fn(optim_vars, aux_vars):
#     print(optim_vars)
#     p1, p2 = optim_vars
    
#     g1, g2 = aux_vars

#     odom = g1.between(g2)

#     odo_pred = p1.between(p2)

#     relative_pose = odom.local(odo_pred)

#     return relative_pose.norm()


In [13]:
# optim_vars = pred_poses[0], pred_poses[1]
# aux_vars = gt_poses[0], gt_poses[1]
# cost_fn = th.AutoDiffCostFunction(optim_vars, quad_error_fn, dim = 1,cost_weight=th.ScaleCostWeight(2.0, name='cost_weight'), aux_vars=aux_vars, name='se2 between func')

In [14]:
# err = quad_error_fn(optim_vars, aux_vars)

In [15]:
objective.add(odo_cost_f)

In [16]:
objective.cost_functions_for_weights

{<theseus.core.cost_weight.ScaleCostWeight at 0x7f81e5802c10>: [<theseus.embodied.measurements.between.Between at 0x7f81e5823070>]}

In [17]:
objective.aux_vars

OrderedDict([('Odo_1', SE2(tensor=tensor([[1., 0., 1., 0.]]), name=Odo_1)),
             ('Variable__26',
              Variable(tensor=tensor([[1.]]), name=Variable__26))])

In [18]:
objective.optim_vars

OrderedDict([('SE2_1',
              SE2(tensor=tensor([[ 0.5052,  0.0529,  0.2942, -0.9557]]), name=SE2_1)),
             ('SE2_2',
              SE2(tensor=tensor([[ 0.1407,  0.2569,  0.6128, -0.7902]]), name=SE2_2))])

In [19]:
optimizer = th.LevenbergMarquardt(
    objective,
    th.CholeskyDenseSolver,
    max_iterations=50,
    step_size=0.3
)

In [20]:
theseus_optim = th.TheseusLayer(optimizer)

In [21]:
pred_poses =[th.rand_se2(1), th.rand_se2(1)]
pred_poses[0].name = "SE2_1"
pred_poses[1].name = "SE2_2"
pred_poses

theseus_inputs = {
    # "gt_start" : gt_poses[0],
    # "gt_finish" : gt_poses[1],
    "Odo_1" : measurement,
    "SE2_1" : pred_poses[0],
    "SE2_2" : pred_poses[1]
}

In [22]:
theseus_inputs

{'Odo_1': SE2(tensor=tensor([[1., 0., 1., 0.]]), name=Odo_1),
 'SE2_1': SE2(tensor=tensor([[ 0.3063,  0.6231, -0.4794,  0.8776]]), name=SE2_1),
 'SE2_2': SE2(tensor=tensor([[ 0.8779,  0.2087, -0.5828,  0.8126]]), name=SE2_2)}

In [23]:
objective.update(theseus_inputs)

In [24]:
with torch.no_grad():
    updated_inputs, info = theseus_optim.forward(
        theseus_inputs, optimizer_kwargs={"track_best_solution": True, "verbose":True})
print("Best solution:", info.best_solution)

Nonlinear optimizer. Iteration: 0. Error: 1.3961745500564575
Nonlinear optimizer. Iteration: 1. Error: 0.6847780346870422
Nonlinear optimizer. Iteration: 2. Error: 0.33577781915664673
Nonlinear optimizer. Iteration: 3. Error: 0.16462397575378418
Nonlinear optimizer. Iteration: 4. Error: 0.08070533722639084
Nonlinear optimizer. Iteration: 5. Error: 0.03956366330385208
Nonlinear optimizer. Iteration: 6. Error: 0.01939474232494831
Nonlinear optimizer. Iteration: 7. Error: 0.009507530368864536
Nonlinear optimizer. Iteration: 8. Error: 0.00466070557013154
Nonlinear optimizer. Iteration: 9. Error: 0.002284722402691841
Nonlinear optimizer. Iteration: 10. Error: 0.0011199963046237826
Nonlinear optimizer. Iteration: 11. Error: 0.0005490321200340986
Nonlinear optimizer. Iteration: 12. Error: 0.0002691414556466043
Nonlinear optimizer. Iteration: 13. Error: 0.00013193589984439313
Nonlinear optimizer. Iteration: 14. Error: 6.467676575994119e-05
Nonlinear optimizer. Iteration: 15. Error: 3.170491982

In [25]:
t_1 = info.best_solution['SE2_1']
t_2 = info.best_solution['SE2_2']

info.best_solution

{'SE2_1': tensor([[ 0.8361, -0.0195, -0.4812,  0.8766]]),
 'SE2_2': tensor([[ 0.3549,  0.8571, -0.4812,  0.8766]])}

In [26]:
(t_1[:,:2] - t_2[:,:2]).norm()

tensor(1.0000)

In [27]:
objective.aux_vars

OrderedDict([('Odo_1', SE2(tensor=tensor([[1., 0., 1., 0.]]), name=Odo_1)),
             ('Variable__26',
              Variable(tensor=tensor([[1.]]), name=Variable__26))])